In [15]:
from dotenv import load_dotenv
import os

load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN')



In [16]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [17]:
from pinecone import Pinecone, ServerlessSpec
index_name="hybrid-search-langchain-pinecone"

pc=Pinecone(api_key=PINECONE_API_KEY)

if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,
        metric="dotproduct",
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )


In [18]:
# from langchain_community.embeddings import OllamaEmbeddings

# embeddings = OllamaEmbeddings(
#     model="all-minilm"
# )

# embeddings

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

c:\Users\chris\OneDrive\Desktop\Gen-ai\Langchain\Hybrid Search\nevn\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


In [19]:
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [20]:
from pinecone_text.sparse import BM25Encoder

bm25=BM25Encoder().default()


In [ ]:
sentences = [
    "The quick brown fox jumps over the lazy dog",
    "In 2024, I visited New Orleans and enjoyed the jazz music on Bourbon Street late into the night.",
    "The sun is shining in the sky, but rain clouds are gathering over the distant mountains.",
    "In 2025, I am going to visit New York to attend a major artificial intelligence conference.",
    "Quantum computing has the potential to revolutionize cryptography by solving problems intractable for classical computers.",
    "Although the economy showed signs of recovery, inflationary pressures remained a significant concern for policymakers.",
    "During the medieval period, manuscripts were painstakingly copied by hand in monasteries before the invention of the printing press.",
    "Climate change is expected to intensify the frequency of extreme weather events such as hurricanes, droughts, and heat waves.",
    "The spacecraft successfully entered Mars’ orbit, sending back high-resolution images of the planet’s surface.",
    "Philosophers have debated for centuries whether free will truly exists or whether our actions are determined by prior causes.",
    "Machine learning models often require large datasets to generalize effectively and avoid overfitting to training examples.",
    "Despite the protagonist’s flaws, the novel portrays her journey of self-discovery with remarkable depth and nuance."
]


In [35]:


bm25.fit(sentences)
bm25.dump("b25_vals.json")

bm25=BM25Encoder().load("b25_vals.json")


100%|██████████| 4/4 [00:00<00:00, 3996.48it/s]


In [36]:
index = pc.Index(index_name)
retriver=PineconeHybridSearchRetriever(
    embeddings=embeddings,
    sparse_encoder=bm25,
    index=index
)

retriver

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000001AB020EE0B0>, index=<pinecone.db_data.index.Index object at 0x000001AB020EE5F0>)

In [37]:
retriver.add_texts(sentences)

100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [ ]:
test_queries = [
    "Which city did I visit in 2025?",
    "Where am I planning to travel for an AI event?",
    "What type of music is popular in New Orleans?",
    "Which technology could disrupt cryptography?",
    "What do philosophers debate about free will?",
    "Which events are linked to climate change in the text?"
]

for i in test_queries:
    print(retriver.invoke(i))


TypeError: string indices must be integers